# Noteboook for testing view classification

In [ ]:
import sys
import os
import pandas as pd

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))


In [ ]:
import random
import sys

import os
import subprocess
import time
import cv2
import pydicom
from optparse import OptionParser
from shutil import rmtree

import tensorflow as tf
import numpy as np
from scipy.misc import imread

from d00_utils.dcm_utils import dcmdir_to_jpgs_for_classification


In [ ]:
directory = os.path.expanduser('~/data/01_raw/test')

In [ ]:
imagedict = {}
predictions = {}
for filename in os.listdir(directory):
    if "jpg" in filename:
        image = imread(directory + filename, flatten=True).astype("uint8")
        imagedict[filename] = [image.reshape((224, 224, 1))]

tf.reset_default_graph()
sess = tf.Session()
model = vgg.Network(0.0, 0.0, feature_dim, label_dim, False)
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.restore(sess, model_name)

for filename in imagedict:
    predictions[filename] = np.around(
        model.probabilities(sess, imagedict[filename]), decimals=3
    )

return predictions
